In [1]:
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
)
from codebase.plot import plot_density, plot_histogram, plot_line, get_post_df
from codebase.ibis import post_process_sign, exp_and_normalise
import altair as alt
from codebase.classes_data import Data
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from pdb import set_trace

alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [2]:
log_dir1 = 'log/20210301_123541_mcmc_debug/'
exp_data1 = load_obj( 'complete_data', log_dir1)
ps_mcmc = load_obj('mcmc_post_samples', log_dir1)


In [40]:
# log_dir3 = 'log/20210301_135005_rash02//'
log_dir3 = 'log/20210301_151414_rash03//'
# log_dir3 = 'log/20210301_164333_rash04/'
# log_dir3 = 'log/20210301_180804_rash05/'
# log_dir3 = 'log/20210301_190258_rash06/'
# log_dir3 = 'log/20210301_222345_rash07/'
exp_data3 = load_obj('data', log_dir3)
t = load_obj('t', log_dir3)
print(t)


99


In [41]:
exp_data3.raw_data['beta']

array([1., 1., 1., 1., 1., 1.])

## Plot weighted histograms

In [42]:
particles = load_obj('particles', log_dir3)
ps_smc2 = dict()
for name in particles.param_names:
    ps_smc2[name] = particles.extract_particles_in_numpy_array(name)
particles.weights

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [43]:
particles.get_acceptance_rate_for_particle_m(0)

array([1.        , 1.        , 0.42105263, 1.        , 0.75675676,
       1.        , 1.        , 0.68571429, 0.97142857, 0.47058824,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       0.80645161, 1.        , 0.93103448, 0.62068966, 1.        ,
       0.57142857, 0.75      , 0.60714286, 0.81481481, 0.74074074,
       0.77777778, 0.69230769, 0.92307692, 0.96153846, 0.6       ,
       0.88      , 0.72      , 0.66666667, 1.        , 0.95833333,
       1.        , 0.43478261, 0.73913043, 1.        , 1.        ,
       0.90909091, 1.        , 0.7       , 1.        , 0.95      ,
       1.        , 1.        , 1.        , 0.94444444, 1.        ,
       0.33333333, 1.        , 0.94117647, 1.        , 0.76470588,
       0.9375    , 1.        , 1.        , 1.        , 0.71428571,
       1.        , 0.5       , 1.        , 1.        , 0.75      ,
       1.        , 0.83333333, 0.91666667, 1.        , 1.        ,
       0.54545455, 1.        , 1.        , 0.8       , 0.8    

In [44]:
corrs = load_obj('jitter_corrs', log_dir3)
corrs

{'beta': array([-2.50046874e-02,  9.47193244e-02,  0.00000000e+00,  1.83553963e-16,
        -4.50273252e-02,  0.00000000e+00, -2.13267614e-02,  0.00000000e+00,
         7.15163073e-01,  0.00000000e+00,  8.29233425e-01,  1.60731400e-01,
         0.00000000e+00,  2.43554047e-01,  0.00000000e+00,  3.37073095e-02,
         1.95034935e-02,  0.00000000e+00,  0.00000000e+00,  7.15247203e-01,
         0.00000000e+00,  0.00000000e+00,  6.71579985e-17,  0.00000000e+00,
         0.00000000e+00,  3.45328492e-01,  0.00000000e+00,  0.00000000e+00,
         8.84593446e-02,  0.00000000e+00,  0.00000000e+00,  5.45045501e-01,
         0.00000000e+00,  0.00000000e+00,  7.88391267e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  6.45166008e-01,
         6.65518852e-01,  5.78971419e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  5.43942395e-01,  3.86207996e-01,
         0.00000000e+00,  0.00000000e+00,             nan,  0.00000000e+00,
    

In [45]:
# particles = load_obj('particles', log_dir3)
# particles.resample_particles_bundles()
# ps_smc2 = dict()
# for name in particles.param_names:
#     ps_smc2[name] = particles.extract_particles_in_numpy_array(name)


##

## Or load existing directory

In [46]:
ps_mcmc = post_process_sign(ps_mcmc)
ps_smc2 = post_process_sign(ps_smc2)


## Plot MCMC samples

In [47]:
param = 'beta'
df = get_post_df(ps_mcmc[param]) 
df = df[df.row==0]
df['source'] = 'mcmc'


df3 = get_post_df(ps_smc2[param]) 
df3 = df3[df3.row==0]
df3['source'] = 'smc2'

plot_histogram(pd.concat([df, df3]), height=100) 

alt.Chart(...)

In [48]:
param = 'alpha'
df = get_post_df(ps_mcmc[param]) 
df['source'] = 'mcmc'


df3 = get_post_df(ps_smc2[param]) 
df3['source'] = 'smc2'

plot_density(pd.concat([df, df3]), height=100)

alt.Chart(...)

## Plot quantiles

In [49]:
param = 'beta'
df = get_post_df(ps_smc2[param])
df_quant = df.groupby(['row', 'col'])[['value']].quantile(0.025).reset_index()
df_quant.rename({'value':'q1'}, axis=1, inplace=True)
df_quant2 = df.groupby(['row', 'col'])[['value']].quantile(0.975).reset_index()
df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

df = df_quant.merge(df_quant2, on=['row', 'col'])

# simple quantile chart
df['source'] = 'smc2'
c1 = alt.Chart(df[df.row==0]).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None),
        alt.Row('row'),
        alt.Column('col'),
        alt.Color('source')
)
c1

alt.Chart(...)

## chart overlaying quantiles with real data

In [50]:
df['index'] = 'r_' + df.row.astype(str)+'.c_'+df.col.astype(str)
df = df.loc[:,['index', 'q1', 'q2']]
df['source'] = 'smc2'
dd = pd.DataFrame(exp_data1.raw_data['beta'], columns=['data'])
dd['col'] = 0
dd['row'] = np.arange(6)
dd['index'] = 'r_' + dd.row.astype(str)+'.c_'+dd.col.astype(str)
dd = dd.loc[:,['index', 'data']]
plot_data = df.merge(dd, on=['index'])

In [51]:
plot_data = plot_data[plot_data['index']=='r_0.c_0']

c1 = alt.Chart(plot_data).mark_bar(opacity=0.6).encode(
    alt.X('q1', title=None,  scale=alt.Scale(domain=[-2,2])),
    alt.X2('q2', title=None),
    alt.Color('source'),    
)
    

c2 = alt.Chart(plot_data).mark_point(opacity=1, color='red').encode(
        alt.X('data', title=None),
)
(c1+c2).facet(
       'index',
    columns=1
    )


alt.FacetChart(...)


## chart overlaying quantiles with real data

In [37]:
param = 'alpha'
df = get_post_df(ps_smc2[param])
df_quant = df.groupby(['row', 'col'])[['value']].quantile(0.025).reset_index()
df_quant.rename({'value':'q1'}, axis=1, inplace=True)
df_quant2 = df.groupby(['row', 'col'])[['value']].quantile(0.975).reset_index()
df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

df = df_quant.merge(df_quant2, on=['row', 'col'])

# simple quantile chart
df['source'] = 'smc2'
c1 = alt.Chart(df).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None),
        alt.Row('row'),
        alt.Column('col'),
        alt.Color('source')
)
c1

alt.Chart(...)

In [38]:
df['index'] = 'r_' + df.row.astype(str)+'.c_'+df.col.astype(str)
df = df.loc[:,['index', 'q1', 'q2']]
df['source'] = 'smc2'
dd = pd.DataFrame(exp_data1.raw_data['alpha'], columns=['data'])
dd['col'] = np.arange(6)
dd['row'] = 0
dd['index'] = 'r_' + dd.row.astype(str)+'.c_'+dd.col.astype(str)
dd = dd.loc[:,['index', 'data']]
plot_data = df.merge(dd, on=['index'])

In [39]:
c1 = alt.Chart(plot_data).mark_bar(opacity=0.6).encode(
    alt.X('q1', title=None,  scale=alt.Scale(domain=[-3,3])),
    alt.X2('q2', title=None),
    alt.Color('source'),    
)
    

c2 = alt.Chart(plot_data).mark_point(opacity=1, color='red').encode(
        alt.X('data', title=None),
)
(c1+c2).facet(
       'index',
    columns=1
    )


alt.FacetChart(...)